## 🐍 파이썬 텍스트 분석: 고전적 벡터화 기법 실습 과제

이 노트북은 '고전적 벡터화 기법 마스터하기' 튜토리얼의 연습 문제와 최종 실습 프로젝트를 포함합니다.

### 필요 라이브러리 설치 및 임포트

In [ ]:
!pip install kiwipiepy scikit-learn pandas gensim plotly_express

In [ ]:
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec, FastText
import plotly.express as px
from sklearn.decomposition import PCA

# Kiwi 형태소 분석기 초기화
kiwi = Kiwi()

---

### 1. Bag-of-Words (BoW)와 DTM: 연습 문제

**실습용 데이터**

In [ ]:
corpus = [
    '배우의 연기력이 정말 대단한 영화였어요.',
    '스토리가 너무 예측 가능해서 연기력이 아까웠다.',
    '감독의 연출과 배우의 연기가 조화로웠던 영화.',
    '와 이 영화 진짜 대박이야! 배우들 연기 미쳤고 스토리도 완전 몰입됨',
    '음... 좀 아쉽네요. 감독이 뭘 말하고 싶었는지 모르겠어요',
    '연기는 괜찮았는데 결말이 너무 뻔해서 실망했습니다',
    '헐 이거 완전 꿀잼ㅋㅋ 예상 못한 반전에 소름돋았어',
    '감독님... 제발 좀 더 신경써서 찍으시길... 연출이 엉망이에요',
    '주연배우 연기 진짜 자연스럽더라! 몰입도 최고였음',
    '스토리가 조금 복잡하긴 했지만 나름 볼만했어요',
    '이런 영화를 왜 만들었는지 이해가 안 가네... 시간 아까움',
    '배우들 케미 완전 좋았고 연출도 깔끔했음. 추천!',
    '예측할 수 없는 전개로 끝까지 출긴장감 넘쳤습니다',
    '연기력은 인정하지만 스토리가 너무 뻔해서... 그냥 그래요',
    '감독의 의도는 좋았으나 표현 방식이 아쉬웠네요',
    'ㅋㅋㅋ 이거 뭐야 완전 재밌잖아? 배우들 연기 ㄹㅇ 대단함',
    '조용한 영화인데 배우들 연기가 워낙 좋아서 지루하지 않았어요',
    '액션은 별로였지만 인간관계 드라마가 탄탄해서 만족',
    '아 진짜... 왜 이렇게 만들었을까? 감독 뭐하는 거야',
    '처음엔 지루했는데 중반부터 완전 몰입! 연출 센스 있네',
    '배우들 연기는 좋았지만 전체적으로 밋밋한 느낌이에요',
    '와... 이런 스토리는 처음 봐! 정말 신선하고 감동적이었어',
    '연출과 연기 모두 완벽했습니다. 올해 최고의 작품 중 하나!',
    '뭔가 아쉬운 부분들이 있지만 그래도 볼만한 영화였어요'
]

**문제 1: 토크나이저를 수정하여 일반 명사(`NNG`)만 추출하고 DTM을 생성하세요.**

In [ ]:
# 연습 문제 1번 풀이 공간

# 1. NNG만 추출하는 토크나이저 정의
def nng_tokenizer(text: str) -> list[str]:
    # YOUR CODE HERE
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens if token.tag == 'NNG']

# 2. CountVectorizer 생성 및 DTM 구축
nng_vectorizer = CountVectorizer(tokenizer=nng_tokenizer)
dtm_nng = nng_vectorizer.fit_transform(corpus)

# 3. 결과 확인
feature_names_nng = nng_vectorizer.get_feature_names_out()
dtm_nng_df = pd.DataFrame(dtm_nng.toarray(), columns=feature_names_nng)

print("어휘 사전 (NNG):")
print(feature_names_nng)
print("\nDTM (NNG):")
print(dtm_nng_df)

**문제 2: `ngram_range=(1, 2)`를 사용하여 DTM을 만들고 어휘 사전을 확인하세요.**

* **`ngram_range=(1, 2)`의 의미:** 단어를 1개씩(unigram) 그리고 연속된 2개씩(bigram) 묶어서 토큰으로 사용하겠다는 의미입니다. 예를 들어 '배우 연기'가 토큰화되면, '배우', '연기' (unigrams) 뿐만 아니라 '배우 연기' (bigram)도 하나의 피처로 간주합니다. 이를 통해 단어의 순서 정보를 일부 보존할 수 있습니다.

In [ ]:
# 연습 문제 2번 풀이 공간

# 튜토리얼의 기본 토크나이저를 다시 사용합니다.
def kiwi_tokenizer(text: str) -> list[str]:
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA']]

# 1. ngram_range를 적용하여 CountVectorizer 생성
ngram_vectorizer = CountVectorizer(tokenizer=kiwi_tokenizer, ngram_range=(1, 2))
dtm_ngram = ngram_vectorizer.fit_transform(corpus)

# 2. 결과 확인
feature_names_ngram = ngram_vectorizer.get_feature_names_out()

print("어휘 사전 (ngram_range=(1, 2)):")
print(feature_names_ngram)

dtm_ngram_df = pd.DataFrame(dtm_ngram.toarray(), columns=feature_names_ngram)
print("\nDTM (ngram_range=(1, 2)):")
print(dtm_ngram_df)

---

### 2. TF-IDF: 연습 문제

**문제 1: 학습된 `TfidfVectorizer`의 `idf_` 속성을 확인하고, IDF 값이 가장 높은 단어와 낮은 단어를 찾아 그 이유를 설명하세요.**

In [ ]:
# 연습 문제 1번 풀이 공간

# 튜토리얼 예제 코드 재사용
tfidf_vectorizer = TfidfVectorizer(tokenizer=kiwi_tokenizer)
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 1. 단어와 IDF 값을 묶어서 DataFrame 생성
idf_df = pd.DataFrame({
    'word': tfidf_vectorizer.get_feature_names_out(),
    'idf': tfidf_vectorizer.idf_
})

idf_df_sorted = idf_df.sort_values('idf', ascending=False)

print(idf_df_sorted)

print("\n--- 분석 ---")
print("가장 IDF가 높은 단어들(가장 희귀한 단어들): '감독', '대단', '스토리' 등. 이 단어들은 단 하나의 문서에만 등장했기 때문입니다.")
print("가장 IDF가 낮은 단어들(가장 흔한 단어들): '배우', '연기', '영화'. 이 단어들은 여러 문서에 걸쳐 등장했기 때문입니다.")

**문제 2: `max_df=0.8`, `min_df=2` 파라미터를 추가하여 TF-IDF 행렬을 다시 만들고 어휘 사전의 변화를 확인하세요.**

* `max_df=0.8`: 단어가 전체 문서의 80%를 초과하여 나타나면 어휘 사전에서 제외합니다. 너무 흔한 단어를 제거하는 역할을 합니다. (0~1 사이의 float 값은 비율, 정수 값은 문서 수를 의미합니다.)
* `min_df=2`: 단어가 최소 2개 이상의 문서에 나타나야만 어휘 사전에 포함시킵니다. 너무 희귀한 단어나 오탈자를 제거하는 효과가 있습니다.

In [ ]:
# 연습 문제 2번 풀이 공간

# 1. max_df, min_df 파라미터를 적용하여 TfidfVectorizer 생성
tfidf_vectorizer_filtered = TfidfVectorizer(
    tokenizer=kiwi_tokenizer,
    max_df=0.8,
    min_df=2
)
tfidf_matrix_filtered = tfidf_vectorizer_filtered.fit_transform(corpus)

# 2. 결과 확인
feature_names_filtered = tfidf_vectorizer_filtered.get_feature_names_out()

print("필터링된 어휘 사전:")
print(feature_names_filtered)

print("\n--- 분석 ---")
print("min_df=2 조건에 따라, 2개 미만의 문서에 등장한 '감독', '대단', '스토리', '아깝다', '예측', '연출', '조화', '가능' 등의 단어들이 모두 제외되었습니다.")
print("'배우', '연기력', '영화'는 2개 이상의 문서에 등장했으므로 어휘 사전에 남았습니다.")
print("(max_df=0.8은 3개 문서 중 80%인 2.4개를 초과하는, 즉 3개 문서 모두에 나타나는 단어를 제외하는데, 현재 그런 단어가 없으므로 이 조건은 아무런 영향을 주지 않았습니다.)")

---

### 3 & 4. Word2Vec & FastText: 연습 문제

**실습용 데이터**

In [ ]:
# 명사, 동사, 형용사만 추출
tokenized_corpus = [
    [token.form for token in kiwi.tokenize(doc) if token.tag in ['NNG', 'NNP', 'VV', 'VA']]
    for doc in corpus
]

**Word2Vec 문제 1: `vector_size=200`, `window=5`로 변경하여 모델을 만들고 '영화'와 유사한 단어를 찾으세요.**

In [ ]:
# 연습 문제 1번 풀이 공간 (Word2Vec)

model_w2v_2 = Word2Vec(sentences=tokenized_corpus, vector_size=200, window=5, min_count=1, sg=1)

try:
    similar_to_movie = model_w2v_2.wv.most_similar('영화', topn=3)
    print("'영화'와 가장 유사한 단어 (vector_size=200, window=5):", similar_to_movie)
except KeyError as e:
    print(f"오류: '{e.args[0]}' 단어가 어휘 사전에 없습니다.")

**Word2Vec 문제 2: '연기' vs '연출', '연기' vs '스토리' 유사도를 비교 분석하세요.**

In [ ]:
# 연습 문제 2번 풀이 공간 (Word2Vec)
model_w2v_1 = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=3, min_count=1, sg=1)

try:
    sim_acting_directing = model_w2v_1.wv.similarity('연기', '연출')
    sim_acting_story = model_w2v_1.wv.similarity('연기', '스토리')

    print(f"'연기' vs '연출' 유사도: {sim_acting_directing:.4f}")
    print(f"'연기' vs '스토리' 유사도: {sim_acting_story:.4f}")
    print("\n--- 분석 ---")
    print("학습 데이터가 작아 결과가 매번 달라질 수 있지만, '연기'와 '연출'이 '연기'와 '스토리'보다 더 유사하게 나올 가능성이 높습니다.")
    print("'연기'와 '연출'은 '배우', '감독' 등과 함께 영화의 제작/표현 요소를 구성하므로 더 강한 문맥적 연관성을 가질 수 있습니다.")
except KeyError as e:
    print(f"오류: '{e.args[0]}' 단어가 어휘 사전에 없습니다.")


**FastText 문제 1: OOV 단어 '시나리오'의 유사 단어를 찾아보세요.**

In [ ]:
# 연습 문제 1번 풀이 공간 (FastText)

ft_model = FastText(sentences=tokenized_corpus, vector_size=100, window=3, min_count=1, sg=1)

oov_word = '시나리오'

try:
    similar_to_oov = ft_model.wv.most_similar(oov_word)
    print(f"OOV 단어 '{oov_word}'와 유사한 단어: {similar_to_oov}")
    print("\n--- 분석 ---")
    print("'시나리오'는 학습 말뭉치에 없지만, 이를 구성하는 '시나리', '나리오' 등의 문자 n-gram 정보를 통해 '스토리', '구성'과 유사한 단어들과 가깝게 벡터가 형성된 것을 볼 수 있습니다.")
except Exception as e:
    print(f"FastText 처리 중 에러 발생: {e}")

---

### 💼 실습 프로젝트: 영화 리뷰 감성 분석 및 핵심어 시각화

**데이터 다운로드**

In [ ]:
# Naver Sentiment Movie Corpus 다운로드
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt -O ratings_train.txt

**1. 데이터 로드 및 전처리**

In [ ]:
# 데이터 로드 (탭으로 구분되어 있음)
df = pd.read_csv('ratings_train.txt', sep='\t')

# 결측치 제거
df.dropna(inplace=True)

# 분석을 위해 데이터 1000개 샘플링 (실제 프로젝트에서는 전체 데이터 사용)
df_sample = df.sample(n=1000, random_state=42)

# kiwipiepy 토크나이저 (명사, 동사, 형용사, 부사 추출)
def nsmc_tokenizer(text):
    tokens = kiwi.tokenize(text)
    return [token.form for token in tokens if token.tag in ['NNG', 'NNP', 'VV', 'VA', 'MAG']]

# X (리뷰), y (라벨) 분리
X = df_sample['document']
y = df_sample['label']

**2. TF-IDF 기반 모델링 및 핵심어 추출**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# TF-IDF 벡터화
tfidf_vec = TfidfVectorizer(tokenizer=nsmc_tokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)
X_tfidf = tfidf_vec.fit_transform(X)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습 및 평가
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(f"TF-IDF 기반 로지스틱 회귀 정확도: {accuracy_score(y_test, y_pred):.4f}")

# 긍정/부정 핵심어 추출
feature_names = tfidf_vec.get_feature_names_out()
coef_df = pd.DataFrame({'word': feature_names, 'coef': lr.coef_.flatten()})

print("\n--- 긍정 리뷰 핵심어 TOP 10 ---")
print(coef_df.sort_values('coef', ascending=False).head(10))

print("\n--- 부정 리뷰 핵심어 TOP 10 ---")
print(coef_df.sort_values('coef', ascending=True).head(10))

**3. BM25 기반 모델링 및 관련도 추출**

In [ ]:
from rank_bm25 import BM25Okapi

# BM25를 위한 문서 토큰화
tokenized_docs = [nsmc_tokenizer(doc) for doc in X]

# BM25 객체 생성
bm25 = BM25Okapi(tokenized_docs)

# 테스트 질의 정의
queries = ["재미있는 영화", "지루한 스토리", "훌륭한 연기"]

print("=== BM25 기반 문서 검색 결과 ===\n")

for query in queries:
    # 질의 토큰화
    tokenized_query = nsmc_tokenizer(query)
    print(f"질의: '{query}'")
    print(f"토큰화된 질의: {tokenized_query}")
    
    # BM25 점수 계산
    doc_scores = bm25.get_scores(tokenized_query)
    
    # 상위 5개 문서 인덱스 추출
    top_docs = np.argsort(doc_scores)[::-1][:5]
    
    print(f"\n상위 5개 관련 문서:")
    for i, doc_idx in enumerate(top_docs):
        print(f"{i+1}. 문서 {doc_idx} (점수: {doc_scores[doc_idx]:.4f})")
        print(f"   내용: {X.iloc[doc_idx][:100]}...")
        print(f"   라벨: {'긍정' if y.iloc[doc_idx] == 1 else '부정'}")
        print()
    
    print("-" * 80)
    print()


**4. (심화) Word2Vec 임베딩 학습 및 시각화**

In [ ]:
# 전체 샘플 데이터 토큰화
tokenized_nsmc = [nsmc_tokenizer(doc) for doc in X]

# Word2Vec 모델 학습
w2v_nsmc_model = Word2Vec(sentences=tokenized_nsmc, vector_size=100, window=5, min_count=5, workers=4, sg=1)

# 주요 단어 및 유사어 리스트
target_words = ['영화', '재미', '최고', '배우', '감동', '쓰레기', '지루하다']
vocab_to_show = []
for word in target_words:
    if word in w2v_nsmc_model.wv:
        vocab_to_show.append(word)
        vocab_to_show.extend([w for w, s in w2v_nsmc_model.wv.most_similar(word, topn=5)])

vocab_to_show = list(set(vocab_to_show)) # 중복 제거

# 시각화를 위한 단어 벡터 추출
word_vectors = np.array([w2v_nsmc_model.wv[word] for word in vocab_to_show])

# PCA로 2차원 축소
pca = PCA(n_components=2)
pca_result = pca.fit_transform(word_vectors)

pca_df = pd.DataFrame(pca_result, columns=['x', 'y'])
pca_df['word'] = vocab_to_show

# Plotly Express로 시각화
fig = px.scatter(pca_df, x='x', y='y', text='word', title='Word2Vec 임베딩 시각화 (PCA)')
fig.update_traces(textposition='top center')
fig.show()